In [2]:
import pandas as pd
import requests

# This notebook is used to demonstrate Thalex's MQP rewards

We start by pulling the orderbook data for BTC-PERPETUAL from the Thalex API


In [3]:
# Define the API endpoint
url = "https://thalex.com/api/v2/public/book"

# Define the query parameters
params = {"instrument_name": "BTC-PERPETUAL"}

# Make the GET request
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Extract the order book data
    order_book = data["result"]

    # Print the order book data
    print("Bids:", order_book["bids"])
    print("Asks:", order_book["asks"])
    print("Last traded price:", order_book["last"])
    print("Timestamp:", order_book["time"])
else:
    print(f"Error: {response.status_code}")
    print(response.text)


def fetch_orderbook(api_url, params):
    response = requests.get(api_url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data["result"]
    else:
        print(f"Error fetching orderbook: {response.status_code}")
        print(response.text)
        return None

Bids: [[98430, 10.047, 10.047], [98417, 9.697, 9.697], [98412, 0.051, 0.051], [98394, 0.002, 0.002], [98390, 0.142, 0]]
Asks: [[98497, 10, 10], [98498, 9.931, 9.931], [98505, 0.051, 0.051], [98520, 0.211, 0.211], [98540, 0.2, 0.2]]
Last traded price: 98301
Timestamp: 1738851143.5465353


### Next, we step the maximum and minium TOBE thresholds and rewards as defined in the MQP document


In [5]:
# Thresholds for the total TOBE of the order book
min_threshold, max_threshold = 0.1, 3.5
max_rewards, min_rewards = 37500, 0

In [6]:
print(f"TOBE min: {min_threshold}, TOBE max: {max_threshold}")


def calculate_monthly_rewards(order_tobe, total_tobe):
    # If total TOBE is below the minimum threshold, no rewards are given
    if total_tobe < min_threshold:
        return 0

    # If total TOBE exceeds the maximum threshold, rewards are maxed out
    if total_tobe >= max_threshold:
        reward_allocation = max_rewards
    else:
        # Linear scaling of rewards based on total TOBE
        reward_range = max_rewards - min_rewards
        tobe_range = max_threshold - min_threshold
        reward_allocation = min_rewards + (total_tobe - min_threshold) * (
            reward_range / tobe_range
        )

    # Distribute the rewards proportionally to each order's TOBE
    return (order_tobe / total_tobe) * reward_allocation


def calculate_tobe_metrics(book, index_price, base, bps, show_customer=False):
    orders = []
    best_bid = max([price for price, _, _ in book["bids"]])
    best_ask = min([price for price, _, _ in book["asks"]])
    mid_price = (best_bid + best_ask) / 2
    typ_dist = bps * 10**-4 * index_price
    print(f"Mid Price: ${mid_price:,.2f}")

    customer_tobe = {}

    for level, (price, quantity, customer) in enumerate(book["asks"]):
        price_distance = abs(mid_price - price)
        normalized_distance = price_distance / typ_dist
        price_score = base**normalized_distance
        tobe_score = quantity * price_score
        tobe_score_new = tobe_score if tobe_score <= 0.5 else 0.5
        customer_tobe[customer] = customer_tobe.get(customer, 0) + tobe_score
        orders.append(
            {
                "Level": level,
                "Side": "ask",
                "Price (USD)": price,
                "Size (BTC)": quantity,
                "Customer": customer,
                "PD": price_distance,
                "ND": normalized_distance,
                "PS": price_score,
                "TOBE": tobe_score,
                "TOBE_New": tobe_score_new,
            }
        )

    for level, (price, quantity, customer) in enumerate(book["bids"]):
        price_distance = abs(mid_price - price)
        normalized_distance = price_distance / typ_dist
        price_score = base**normalized_distance
        tobe_score = quantity * price_score
        tobe_score_new = tobe_score if tobe_score <= 0.5 else 0.5
        customer_tobe[customer] = customer_tobe.get(customer, 0) + tobe_score
        orders.append(
            {
                "Level": level,
                "Side": "bid",
                "Price (USD)": price,
                "Size (BTC)": quantity,
                "Customer": customer,
                "PD": price_distance,
                "ND": normalized_distance,
                "PS": price_score,
                "TOBE": tobe_score,
                "TOBE_New": tobe_score_new,
            }
        )

    # Convert the list of orders to a DataFrame
    df = pd.DataFrame(orders)

    # Drop the specified columns
    columns_to_drop = ["PD", "ND", "PS", "MQS"]
    df = df.drop(
        columns=columns_to_drop, errors="ignore"
    )  # Safeguard with 'ignore' to avoid issues if columns don't exist

    if not show_customer:
        df = df.drop(columns=["Customer"])  # Hide the 'Customer' column if not needed

    total_tobe = df["TOBE"].sum()
    total_tobe_new = df["TOBE_New"].sum()

    # Calculate Monthly Rewards for each order
    df["Monthly_Rewards"] = df.apply(
        lambda row: calculate_monthly_rewards(row["TOBE"], total_tobe), axis=1
    )

    df["Monthly_Rewards_New"] = df.apply(
        lambda row: calculate_monthly_rewards(row["TOBE_New"], total_tobe_new), axis=1
    )

    df["MQS"] = (df["TOBE"] / total_tobe) * 100
    df["MQS_New"] = (df["TOBE_New"] / total_tobe_new) * 100

    df_asks = df[df["Side"] == "ask"].sort_values("Price (USD)", ascending=False)
    df_bids = df[df["Side"] == "bid"].sort_values("Price (USD)", ascending=False)
    df = pd.concat([df_asks, df_bids]).reset_index(drop=True)

    bids_tobe = df[df["Side"] == "bid"]["TOBE"].sum()
    asks_tobe = df[df["Side"] == "ask"]["TOBE"].sum()

    print(f"Bids TOBE sum: {bids_tobe:.2f}")
    print(f"Asks TOBE sum: {asks_tobe:.2f}")
    print(f"Total TOBE sum: {total_tobe:.2f}")
    print(f"Total TOBE sum New: {total_tobe_new:.2f}\n")

    # Calculate and print total monthly rewards
    total_monthly_rewards = df["Monthly_Rewards"].sum()
    print(f"Total Monthly Rewards: ${total_monthly_rewards:,.2f}")
    total_monthly_rewards_new = df["Monthly_Rewards_New"].sum()
    print(f"Total Monthly Rewards New: ${total_monthly_rewards_new:,.2f}\n")

    # print("Customer TOBE and MQS Totals:")
    # for customer, tobe in customer_tobe.items():
    #     mqs = (tobe / total_tobe) * 100
    #     customer_monthly_rewards = calculate_monthly_rewards(tobe, total_tobe)
    #     print(
    #         f"{customer}: TOBE = {tobe:.4f}, MQS = {mqs:.1f}%, Monthly Rewards = ${customer_monthly_rewards:,.2f}"
    #     )

    return df


def format_orderbook(df):
    styles = [
        {"selector": "td", "props": [("border", "0.2px solid grey")]},
        {"selector": "th", "props": [("border", "0.2px solid grey")]},
    ]

    return (
        df.style.bar(subset=["Size (BTC)"], color="#1c4716")
        .background_gradient(subset=["TOBE"], cmap="Blues")
        .set_table_styles(styles)
        .apply(
            lambda s: [
                (
                    "background-color:#691522; color: #e61939; font-weight: bold;"
                    if s == "ask"
                    else "background-color:#1c4716; color: #3fd42a; font-weight: bold;"
                )
                for s in df["Side"]
            ],
            subset=["Price (USD)"],
        )
        .format(
            {
                "Price (USD)": "{:,.1f}",
                "Size (BTC)": "{:,.3f}",
                "TOBE": "{:,.4f}",
                "Monthly_Rewards": "${:,.2f}",
                "TOBE_New": "{:,.4f}",
                "Monthly_Rewards_New": "${:,.2f}",
                "MQS": "{:,.2f}%",
                "MQS_New": "{:,.2f}%",
            }
        )
    )


def analyze_orderbook(api_url, params, index_price, base, bps):
    book = fetch_orderbook(api_url, params)
    if book is None:
        book = fetch_orderbook(api_url, params)
        if book is None:
            print("Failed to fetch orderbook data.")
            return None

    df = calculate_tobe_metrics(book, index_price, base, bps)

    return format_orderbook(df)


index_price = order_book["last"]


TOBE min: 0.1, TOBE max: 3.5


In [7]:
result = analyze_orderbook(url, params, index_price, base=0.5, bps=1)
result

Mid Price: $98,277.00
Bids TOBE sum: 0.46
Asks TOBE sum: 1.80
Total TOBE sum: 2.27
Total TOBE sum New: 0.98

Total Monthly Rewards: $23,883.30
Total Monthly Rewards New: $9,657.92



,Level,Side,Price (USD),Size (BTC),TOBE,TOBE_New,Monthly_Rewards,Monthly_Rewards_New,MQS,MQS_New
0,4,ask,"98,340.0",0.210,0.0025,0.0025,$26.06,$24.46,0.11%,0.25%
1,3,ask,"98,331.0",0.002,0.0000,0.0000,$0.47,$0.44,0.00%,0.00%
2,2,ask,"98,320.0",0.198,0.0095,0.0095,$100.65,$94.50,0.42%,0.98%
3,1,ask,"98,314.0",0.010,0.0007,0.0007,$7.76,$7.29,0.03%,0.08%
4,0,ask,"98,301.0",9.722,1.7898,0.5000,"$18,868.72","$4,949.47",79.00%,51.25%
5,0,bid,"98,253.0",0.047,0.0087,0.0087,$91.22,$85.65,0.38%,0.89%
6,1,bid,"98,233.0",10.063,0.4522,0.4522,"$4,767.04","$4,476.02",19.96%,46.35%
7,2,bid,"98,221.0",0.002,0.0000,0.0000,$0.41,$0.38,0.00%,0.00%
8,3,bid,"98,212.0",0.193,0.0020,0.0020,$20.80,$19.53,0.09%,0.20%
9,4,bid,"98,210.0",0.002,0.0000,0.0000,$0.19,$0.18,0.00%,0.00%
